In [11]:
import pandas as pd

# Step 1: Load the original CSV file
# Replace 'airlines_original.csv' with the path to your CSV file
input_file = 'airlines_original.csv'
output_file = 'airlines_ideal_format.csv'

# Read the CSV file without headers because the structure is custom
df = pd.read_csv(input_file, header=None)

# Step 2: Extract Column Names and Data
# The first row contains metadata labels
column_labels = df.iloc[0, :].tolist()  # Get the first row for column labels
df.columns = column_labels  # Assign these column labels to the DataFrame

# Remove the first row since it's now used as headers
df = df.drop(0).reset_index(drop=True)

# Step 3: Transpose the DataFrame
# Set 'Airline' or 'Attribute' as the index (depending on your structure)
df_transposed = df.set_index('Airline').T.reset_index()

# Step 4: Melt the DataFrame to Long Format
# Identify columns that are dates (assuming they contain a 'Q' pattern)
date_columns = [col for col in df_transposed.columns if 'Q' in col]

# Melt the DataFrame to have 'Date', 'Value' columns
long_df = pd.melt(df_transposed, id_vars=['index'], value_vars=date_columns,
                  var_name='Date', value_name='Value')

# Rename 'index' to 'Airline' since it represents airlines after transposing
long_df.rename(columns={'index': 'Airline'}, inplace=True)

# Step 5: Clean the Data
# Remove dollar signs and commas from 'Value' and convert to numeric
long_df['Value'] = long_df['Value'].replace({'\$': '', ',': ''}, regex=True)
long_df['Value'] = pd.to_numeric(long_df['Value'], errors='coerce')

# Remove any rows with missing or zero values if necessary
long_df = long_df.dropna(subset=['Value'])
long_df = long_df[long_df['Value'] != 0]

# Step 6: Add Metadata Columns Back
# Extract metadata columns like 'Status', 'Country', 'Region', etc.
metadata_cols = ['Airline', 'Status', 'Country', 'Region', 'IATA']
metadata = df[metadata_cols].drop_duplicates()

# Merge metadata back into the long DataFrame
final_df = pd.merge(long_df, metadata, on='Airline', how='left')

# Step 7: Save the Data to a New CSV File
final_df.to_csv(output_file, index=False)

print("Data cleaning and formatting complete. Output saved to:", output_file)